<a href="https://colab.research.google.com/github/zifeiYv/BiLSTM-CRF/blob/master/%E5%8F%82%E8%80%83%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1、Imports

refs: https://www.kaggle.com/code/dimitriosroussis/electricity-price-forecasting-with-dnns-eda/notebook

In [ ]:
from collections import deque

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import statsmodels.api as sm
import seaborn as sns

from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller, kpss, ccf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
all_df = pd.read_csv('/content/drive/MyDrive/风行数据/all_df.csv', parse_dates=['时间戳'])

# 2、数据探索

In [ ]:
# 价格与周平均价格趋势变化图
rolling = all_df['统一出清价格-日前(元/MWh)'].rolling(96*7, center=True).mean()

fig, ax = plt.subplots(figsize=(30, 12))
ax.set_xlabel('Time', fontsize=16)
ax.plot(all_df['时间戳'], all_df['统一出清价格-日前(元/MWh)'], label='real time price')
ax.plot(all_df['时间戳'], rolling, label='week mean price', linestyle='-', linewidth=2)

ax.legend()
plt.show()

In [ ]:
# 价格滞后（一日）比率图
change = all_df['统一出清价格-日前(元/MWh)'].div(all_df['统一出清价格-日前(元/MWh)'].shift(96)).mul(100)
fig, ax = plt.subplots(figsize=(30, 12))
ax.plot(change[:96*7])
plt.show()

In [ ]:
# 价格分布直方图
all_df['统一出清价格-日前(元/MWh)'].plot.hist(bins=18, alpha=0.65)

In [ ]:
# 价格的季节分解图
res = sm.tsa.seasonal_decompose(all_df['统一出清价格-日前(元/MWh)'], model='additive', freq=4*24)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(20, 12))
res.observed.plot(ax=ax1, title='Observed')
res.trend.plot(ax=ax2, title='Trend')
res.resid.plot(ax=ax3, title='Residual')
res.seasonal.plot(ax=ax4, title='Seasonal')
plt.tight_layout()
plt.show()

In [ ]:
# 对价格进行ADF检验
# 原假设：时间序列是非平稳的，存在单位根
# 备择假设：时间序列是平稳的
# 
adf_test = adfuller(all_df['统一出清价格-日前(元/MWh)'], regression='c')
print('ADF Statistic: {:.6f}\np-value: {:.6f}\n#Lags used: {}'
      .format(adf_test[0], adf_test[1], adf_test[2]))
for key, value in adf_test[4].items():
    print('Critical Value ({}): {:.6f}'.format(key, value))
# 检验统计量的值小于1%水平下的临界值，说明有99%的把握拒绝原假设，即价格序列可以认为是平稳的

In [ ]:
# 对价格进行KPSS检验
# 原假设：时间序列在一个常数附近是平稳的
# 备择假设：时间序列存在一个单位根，即，是不平稳的
kpss_test = kpss(all_df['统一出清价格-日前(元/MWh)'], regression='c', lags='legacy')
print('KPSS Statistic: {:.6f}\np-value: {:.6f}\n#Lags used: {}'
      .format(kpss_test[0], kpss_test[1], kpss_test[2]))
for key, value in kpss_test[3].items():
    print('Critical Value ({}): {:.6f}'.format(key, value))
# 检验统计量的值大于1%水平下的临界值，说明在此种情况下无法拒绝原假设，即价格序列可以认为是平稳的

In [ ]:
# 绘制自相关与偏自相关图
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10, 6))
plot_acf(all_df['统一出清价格-日前(元/MWh)'], lags=200, ax=ax1)
plot_pacf(all_df['统一出清价格-日前(元/MWh)'], lags=50, ax=ax2)
plt.tight_layout()
plt.show()

In [ ]:
# 相关性矩阵（皮尔森相关系数）
correlations = all_df.corr(method='pearson')
fig = plt.figure(figsize=(24, 24))
sns.heatmap(correlations, annot=True, fmt='.2f')
plt.title('Pearson Correlation Matrix')
plt.show()

# 3、特征工程

In [ ]:
is_peak = []  # 是否用电高峰
hours = []  # 当前时间戳的所属小时
weekdays = []  # 当前日期是周几
months = []  # 当前日期所属的月份
is_weekend = []  # 当前是否为周末

for i in range(len(all_df)):
    timestamp = all_df['时间戳'].iloc[i]

    if 10 <= timestamp.hour <= 15:
            is_peak.append(1)
    else:
        is_peak.append(0)
    wd = timestamp.weekday()
    weekdays.append(wd)
    if wd in [5, 6]:
        is_weekend.append(1)
    else:
        is_weekend.append(0)
    hours.append(timestamp.hour)
    months.append(timestamp.month)

all_df['is_peak'] = is_peak
all_df['is_weekend'] = is_weekend
all_df['小时'] = hours
all_df['周'] = weekdays
all_df['月'] = months

all_df = all_df[['省调负荷-日前(MW)', '时间戳', '火电竞价空间-日前(MW)', '新能源负荷-日前(MW)', '统一出清价格-日前(元/MWh)', '小时', '周', '月', 'is_peak', 'is_weekend']]

# 为了绘图正确显示文字，全部采用英文字段名
columns = ['dispatch load', 'timestamp', 'bid space', 'new energy load', 'price', 'hour', 'weekday', 'month', 'is peak', 'is weekend']
all_df.columns = columns
all_df.set_index('timestamp', inplace=True)
all_df.head()


# 4、建模

先定义一些用于评估、可视化预测结果的功能函数

In [ ]:
def compare_plot(y_true, y_pred, start_index=None, end_index=None):
    start, end = start_index, end_index
    if start is None:
        start = 0
    else:
        start *= 96
    if end is None:
        end = -1
    else:
        end *= 96
    fig, ax = plt.subplots(figsize=(30, 12))
    ax.set_xlabel('Time', fontsize=16)
    ax.plot(y_true[start: end], label='true value')
    ax.plot(y_pred[start: end], label='predict value', linestyle='-', linewidth=1)

    ax.legend()
    plt.show()


def post_process(y_pred):
    ret = []
    for i in y_pred:
        if i < 0:
            ret.append(0)
        elif i > 1500:
            ret.append(1500)
        else:
            ret.append(i)
    return ret


def filtered_mape(y_true: list, y_pred: list):
    """为了使mape的值更加合理，过滤掉其中的0值"""
    y_true_filtered, y_pred_filtered = [], []
    for i in range(len(y_true)):
        if y_true[i] != 0:
            y_true_filtered.append(y_true[i])
            y_pred_filtered.append(y_pred[i])
            
    return mean_absolute_percentage_error(y_true_filtered, y_pred_filtered)

## 4.1 机器学习模型

### 4.1.1 Random Forest

In [ ]:
train_size = int(all_df.shape[0] * 0.8)
test_size = all_df.shape[0] - train_size
print(f"train data size: {train_size}")
print(f"test data size: {test_size}")

train_data = all_df[:train_size]
test_data = all_df[-test_size:]

train_X = train_data[['dispatch load', 'bid space', 'new energy load', 'hour', 'weekday', 'month', 'is peak', 'is weekend']]
train_y = train_data[['price']]
test_X = test_data[['dispatch load', 'bid space', 'new energy load', 'hour', 'weekday', 'month', 'is peak', 'is weekend']]
test_y = test_data[['price']]

In [ ]:
rf = RandomForestRegressor(n_estimators=10, criterion='absolute_error', random_state=123)
rf.fit(train_X, train_y)

In [ ]:
rf_pred = rf.predict(test_X)
rf_pred = post_process(rf_pred)
compare_plot(test_y.values, rf_pred, 0, 3)
print('mape: ', filtered_mape(test_y.values, rf_pred))

### 4.1.2 XGBoost Normal

In [ ]:
model=xgb.XGBRegressor(max_depth=3, n_estimators=100, learning_rate=0.1)
model.fit(train_X, train_y)

In [ ]:
xgb_y_pred = model.predict(test_X)
xgb_y_pred = post_process(xgb_y_pred)
compare_plot(test_y.values, rf_pred, 0, 3)
print('mape: ', filtered_mape(test_y.values, xgb_y_pred))

### 4.1.3 XGBoost

价格的影响具有长期性。因此，在预测未来的价格时，需要考虑历史价格的影响。结合当前电力市场的现状，将每三天的价格数据看做一个待预测的点，这样预测得到的价格也包含了三天的值，可以供日前申报使用。

In [ ]:
def create_dataset(all_df, input_days, output_days=3,
                   target_col='price'):
    """
    从原始数据中处理得到数据集
    all_df: 原始数据集
    input_days: 作为特征的天数
    output_days: 待预测的天数，默认为3
    target_col: 目标列的名称
    """
    in_window = input_days * 96
    out_window = output_days * 96
    in_data = []
    out_data = []
    for i in tqdm(range(len(all_df)-in_window-out_window)):
        in_data.append(all_df.iloc[i: i+in_window, :].values)
        out_data.append(all_df[target_col].iloc[i+in_window: i+in_window+out_window].values)
    return np.array(in_data), np.array(out_data)

In [ ]:
X, y = create_dataset(all_df, 4)

In [ ]:
# 按照7:1:2的比例划分为训练集、验证集和测试集
data_len = X.shape[0]
train_size = int(data_len * .7)
val_size = int(data_len * .1)
test_size = data_len - train_size - val_size

train_X = X[:train_size]
train_y = y[:train_size]
val_X = X[train_size: train_size+val_size]
val_y = y[train_size: train_size+val_size]
test_X = X[train_size+val_size: train_size+val_size+test_size]
test_y = y[train_size+val_size: train_size+val_size+test_size]

In [ ]:
train_X = train_X.reshape(-1, train_X.shape[1] * train_X.shape[2])
val_X = val_X.reshape(-1, val_X.shape[1] * val_X.shape[2])
test_X = test_X.reshape(-1, test_X.shape[1] * test_X.shape[2])

In [ ]:
print(train_X.shape)
print(train_y.shape)

In [ ]:
all_df['price'].iloc[0+96*4: 0+96*4+96*3]

In [ ]:
train_y[0]

In [ ]:
param = {'eta': 0.03, 'max_depth': 180, 
         'subsample': 1, 'colsample_bytree': 0.95, 
         'alpha': 0.1, 'lambda': 0.15, 'gamma': 0.1,
         'objective': 'reg:linear', 'eval_metric': 'rmse', 
         'silent': 1, 'min_child_weight': 0.1, 'n_jobs': -1}

dtrain = xgb.DMatrix(train_X, train_y)
dval = xgb.DMatrix(val_X, val_y)
dtest = xgb.DMatrix(test_X)
eval_list = [(dtrain, 'train'), (dval, 'eval')]

In [ ]:
xgb_model = xgb.train(param, dtrain, 180, eval_list, early_stopping_rounds=3)

In [ ]:
# Load a trained model
# xgb_model = xgb.Booster()
# xgb_model.load_model("/content/drive/MyDrive/models/xgb.model")

forecast = xgb_model.predict(dtest)
forecast = post_process(forecast)
compare_plot(test_y[0], forecast[:288], 0, 3)
print('mape: ', filtered_mape(test_y.values, forecast))

## 4.2 深度学习模型

基于pytorch的深度学习预测模型。


导入所有的依赖：

In [ ]:
import torch

from torch.nn import Sequential, Conv1d, ReLU, Linear, LSTM
from torch.utils.data import Dataset, DataLoader

定义一个用于Early Stopping的类，以防止过拟合。

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

接下来需要对经过特征工程后的数据进行处理，包括标准化以及向量化：

In [ ]:
X = all_df.drop('price', axis=1)
y = all_df[['price']]

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

scaler_X.fit(X)
scaler_y.fit(y)

X_norm = scaler_X.transform(X)
y_norm = scaler_y.transform(y)

ds_norm = np.concatenate((X_norm, y_norm), axis=1)

为了刻画历史对于未来价格的影响，需要将数据进行“序列化”：以过去w天的数据作为输入来预测未来d天的价格。在当前现货市场中，d最大取值只需要为3。又由于日前价格申报每天进行，因此，实际上只需要保证d为1时的准确率即可。

为此，需要定义一个将数据序列化的函数：

In [ ]:
def sequence_data(normed_data, input_days, output_days=3, use_out_features=False):
    """
    将数据按照指定的窗口长度序列化

    normed_data: 经过标准化的原始数据，形状为(m,n)，m为样本数，n为特征数+1，注意最后一列
        为特征列
    input_days: 作为特征的天数，即w
    output_days: 待预测的天数，即d，默认为3
    use_out_features: 是否使用待预测的时间戳范围内的特征作为输入
    """
    in_window = input_days * 96
    out_window = output_days * 96
    in_data = []
    out_data = []
    for i in tqdm(range(len(normed_data)-in_window-out_window)):
        if use_out_features:
            in_data.append(normed_data[:, :][i: i+in_window+out_window, :])
        else:
            in_data.append(normed_data[:, :][i: i+in_window, :])
        out_data.append(normed_data[:, -1][i+in_window: i+in_window+out_window])
        
    return np.array(in_data), np.array(out_data)

In [ ]:
X_seq, y_seq = sequence_data(ds_norm, 3, 1, True)
print()
print(f"Samples' number of sequenced data: {X_seq.shape[0]}")
print(f"Original feature number          : {X_seq.shape[-1]}")
print(f"Input timestep length            : {X_seq.shape[1]}")
print(f"Output timestep length           : {y_seq.shape[-1]}")

接下来，划分训练集、验证集与测试集，并将它们转化为tensor。

在划分时，需要注意：`train_data_size`与`eval_data_size`必须为浮点数；`test_data_size`可以为浮点数，也可以为整数。

- `test_data_size`为浮点数时，必须保证$train\_data\_size+eval\_data\_size+test\_data\_size=1$，即按照指定的比例对数据进行划分。
- `test_data_size`为整数时，表明将数据集的最后`test_data_size`个样本作为测试集，其余的样本按照比例被分为训练集和验证集，因此，须保证$train\_data\_size+eval\_data\_size=1$。

In [ ]:
# # 定义部分 # #
train_data_size = 0.75
eval_data_size = 0.25
test_data_size = 1
# # # #

assert isinstance(train_data_size, float)
assert isinstance(eval_data_size, float)
assert isinstance(test_data_size, (float, int))

data_len = X_seq.shape[0]
if isinstance(test_data_size, float):
    assert train_data_size + eval_data_size + test_data_size == 1
    train_data_size = int(data_len * train_data_size)
    eval_data_size = int(data_len * eval_data_size)
    test_data_size = data_len - train_data_size - eval_data_size
else:
    assert train_data_size + eval_data_size == 1
    data_len -= test_data_size
    train_data_size = int(data_len * train_data_size)
    eval_data_size = data_len - train_data_size

train_X = X_seq[:train_data_size]
train_y = y_seq[:train_data_size]

eval_X = X_seq[train_data_size: train_data_size+eval_data_size]
eval_y = y_seq[train_data_size: train_data_size+eval_data_size]

test_X = X_seq[train_data_size+eval_data_size: train_data_size+eval_data_size+test_data_size]
test_y = y_seq[train_data_size+eval_data_size: train_data_size+eval_data_size+test_data_size]

if torch.cuda.is_available():
    train_X_tensor = torch.tensor(train_X).float().cuda()
    train_y_tensor = torch.tensor(train_y).float().cuda()

    eval_X_tensor = torch.tensor(eval_X).float().cuda()
    eval_y_tensor = torch.tensor(eval_y).float().cuda()

    test_X_tensor = torch.tensor(test_X).float().cuda()
    test_y_tensor = torch.tensor(test_y).float().cuda()
else:
    train_X_tensor = torch.tensor(train_X).float()
    train_y_tensor = torch.tensor(train_y).float()

    eval_X_tensor = torch.tensor(eval_X).float()
    eval_y_tensor = torch.tensor(eval_y).float()

    test_X_tensor = torch.tensor(test_X).float()
    test_y_tensor = torch.tensor(test_y).float()


原始数据处理完成之后，需要继承`Dataset`类来根据数据创建ds对象，以供模型使用：

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataset_type,
                 train_X=None, train_y=None,
                 eval_X=None, eval_y=None,
                 test_X=None):
        self.dataset_type = dataset_type
        self.train_X = train_X
        self.train_y = train_y
        self.eval_X = eval_X
        self.eval_y = eval_y
        self.test_X = test_X
    
    def __len__(self):
        if self.dataset_type == 'train':
            return self.train_X.shape[0]
        elif self.dataset_type == 'eval':
            return self.eval_X.shape[0]
        else:
            return self.test_X.shape[0]
    
    def __getitem__(self, i):
        if self.dataset_type == 'train':
            return self.train_X[i], self.train_y[i]
        elif self.dataset_type == 'eval':
            return self.eval_X[i], self.eval_y[i]
        else:
            return self.test_X[i]

最后，定义用于训练、评估以及预测的函数：

In [ ]:
def train(train_data_loader, model, loss_fn, optimizer, 
          eval_data_loader=None, early_stopping=None):
    num_batches = len(train_data_loader)
    total_loss = 0
    loss_list = []
    model.train()

    for X, y in tqdm(train_data_loader):
        output = model(X)
        loss = loss_fn(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        step_loss = loss.item()
        total_loss += step_loss
        loss_list.append(step_loss)
    avg_loss = total_loss / num_batches
    print(f"  Average train loss in current epoch: {avg_loss}")
    if eval_data_loader is not None:
        print('  Evaluating...')
        if evaluate(model, eval_data_loader, early_stopping) == -1:
            return -1
    return loss_list


def evaluate(model, eval_data_loader, early_stopping):
    model.eval()
    with torch.no_grad():
        eval_loss = []
        for X, y in eval_data_loader:
            output = model(X)
            loss = loss_fn(output, y)
            eval_loss.append(loss.item())
        print(f'  Evaluation loss: {np.average(eval_loss)}')
        if early_stopping is not None:
            early_stopping(np.average(eval_loss), model)
            if early_stopping.early_stop:
                print('Early stopped')
                return -1
        print()


def do_train(model, loss_fn, optimizer, train_data_loader, epoches=10,
             eval_data_loader=None, patience=None):
    """
    执行训练。

    model: 待训练的模型
    loss_fn: 损失函数
    optimizer: 优化器
    train_data_loader: 封装的训练数据
    epoches: 训练的轮数
    eval_data_loader: 封装的验证数据
    patience: 用于早停的耐心值
    """
    if torch.cuda.is_available():
        model.cuda()
    
    if eval_data_loader is None or patience is None:
        early_stopping = None
    else:
        early_stopping = EarlyStopping(patience, verbose=True)
    
    train_losses = []

    for epoch in range(epoches):
        print(f"epoch {epoch+1} in {epoches}:")
        epoch_loss = train(train_data_loader, model, loss_fn, optimizer, eval_data_loader, early_stopping)
        if epoch_loss == -1:
            break
        train_losses.extend(epoch_loss)

    return train_losses


def do_predict(model, test_data_loader):
    preds = []
    with torch.no_grad():
        for X in tqdm(test_data_loader):
            pred = model(X)
            if pred.is_cuda:
                preds.append(pred.cpu().numpy())
            else:
                preds.append(pred.numpy())
    return preds


### 4.2.1 BiLSTM

In [ ]:
train_ds = MyDataset('train', train_X=train_X_tensor, train_y=train_y_tensor)
eval_ds = MyDataset('eval', eval_X=eval_X_tensor, eval_y=eval_y_tensor)
test_ds = MyDataset('test', test_X=test_X_tensor)

train_data_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
eval_data_loader = DataLoader(eval_ds, batch_size=32)
test_data_loader = DataLoader(test_ds, batch_size=1)

In [ ]:
# 定义模型架构
class MyLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.lstm = LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True,
            num_layers=4,
            dropout=0.1
        )
        self.linear = torch.nn.Linear(in_features=hidden_size*2,
                                      out_features=96)
    
    def forward(self, x):
        batch_size = x.shape[0]
        o, _ = self.lstm(x)
        out = self.linear(o[:, -1, :])
        return out

In [ ]:
input_size = train_X_tensor.shape[-1]
model = MyLSTM(input_size, 81)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
train_loss = do_train(model, loss_fn, optimizer, train_data_loader, eval_data_loader=eval_data_loader, patience=3)

In [ ]:
model.load_state_dict(torch.load('checkpoint.pt'))


In [ ]:
pred = do_predict(model, test_data_loader)[0]
y_pred = scaler_y.inverse_transform(pred)[0]

In [ ]:
y_pred = post_process(y_pred)

In [ ]:
if test_y_tensor[0].is_cuda:
    y_true = test_y_tensor[0].cpu().reshape(-1, 1)
else:
    y_true = test_y_tensor[0].reshape(-1, 1)
y_true = scaler_y.inverse_transform(y_true)

In [ ]:
compare_plot(y_true, y_pred)

### 4.2.2 将RF的预测值作为BiLSTM的一个特征

In [ ]:
all_df.drop(all_df.index[-1], inplace=True)
# 只保留最后三天的数据作为预测，其余数据为训练集，先训练一个xgb模型
test_index = -(96 * 3)
test_data = all_df[test_index:]
train_data = all_df[:test_index]

In [ ]:
train_X = train_data[['dispatch load', 'bid space', 'new energy load', 'hour', 'weekday', 'month', 'is peak', 'is weekend']]
train_y = train_data[['price']]
test_X = test_data[['dispatch load', 'bid space', 'new energy load', 'hour', 'weekday', 'month', 'is peak', 'is weekend']]
test_y = test_data[['price']]

rf = RandomForestRegressor(n_estimators=10, criterion='absolute_error', random_state=123)
rf.fit(train_X, train_y)

In [ ]:
train_y_pred = rf.predict(train_X)
xgb_y_pred = post_process(train_y_pred)

In [ ]:
compare_plot(train_y.values, xgb_y_pred, 5, 7)

In [ ]:
test_y_pred = rf.predict(test_X)
test_y_pred = post_process(test_y_pred)
compare_plot(test_y.values, test_y_pred)

In [ ]:
# 给原始序列增加一列，值为rf的预测价格
all_df['rf_pred'] = np.concatenate((train_y_pred, test_y_pred)).reshape(1, -1)[0]
# 接下来执行4.4部分的代码

### 4.2.3 CNN




> 当前效果较差


In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

In [ ]:
train_ds = MyDataset('train', train_X=train_X_tensor, train_y=train_y_tensor)
eval_ds = MyDataset('eval', eval_X=eval_X_tensor, eval_y=eval_y_tensor)
test_ds = MyDataset('test', test_X=test_X_tensor)

train_data_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
eval_data_loader = DataLoader(eval_ds, batch_size=32)
test_data_loader = DataLoader(test_ds, batch_size=1)

In [ ]:
in_shape = train_X_tensor.shape
in_channels = in_shape[1]  # input_channels的值为x的时间步长度

cnn = torch.nn.Sequential(
    Conv1d(in_channels=in_channels, out_channels=48, kernel_size=2, padding='same'),
    ReLU(),
    Conv1d(in_channels=48, out_channels=96, kernel_size=2, padding='same'),
    Flatten(),
    Linear(in_features=96*in_shape[-1], out_features=96),
)

In [ ]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=6e-3, amsgrad=True)
loss_fn = torch.nn.MSELoss()

In [ ]:
train_loss = do_train(cnn, loss_fn, optimizer, train_data_loader, eval_data_loader=eval_data_loader, patience=3)

In [ ]:
cnn.load_state_dict(torch.load('checkpoint.pt'))

In [ ]:
y1 = do_predict(cnn, test_data_loader)[0]

In [ ]:
scaler_y.inverse_transform(y1)

In [ ]:
compare_plot(scaler_y.inverse_transform(test_y_tensor[0].reshape(-1,1)), scaler_y.inverse_transform(y1)[0])